In [16]:
import numpy as np

class Audiobooks_Data_Reader():
    def __init__(self, dataset, batch_size = None):
    
        # The dataset that loads is one of "train", "validation", "test".
        # e.g. if I call this class with x('train',5), it will load 'Audiobooks_data_train.npz' with a batch size of 5.
        npz = np.load('{0}.npz'.format(dataset))
        
        # Two variables that take the values of the inputs and the targets. Inputs are floats, targets are integers
        self.inputs, self.targets = npz['inputs'].astype(np.float), npz['targets'].astype(np.int)
        
        # Counts the batch number, given the size you feed it later
        # If the batch size is None, we are either validating or testing, so we want to take the data in a single batch
        if batch_size is None:
            self.batch_size = self.inputs.shape[0]
        else:
            self.batch_size = batch_size
        self.curr_batch = 0
        self.batch_count = self.inputs.shape[0] // self.batch_size
    
    # A method which loads the next batch
    def __next__(self):
        if self.curr_batch >= self.batch_count:
            self.curr_batch = 0
            raise StopIteration()
            
        # You slice the dataset in batches and then the "next" function loads them one after the other
        batch_slice = slice(self.curr_batch * self.batch_size, (self.curr_batch + 1) * self.batch_size)
        inputs_batch = self.inputs[batch_slice]
        targets_batch = self.targets[batch_slice]
        self.curr_batch += 1
        classes_num = 2
        targets_one_hot = np.zeros((targets_batch.shape[0], classes_num))
        targets_one_hot[range(targets_batch.shape[0]), targets_batch] = 1
        return inputs_batch, targets_one_hot

    def __iter__(self):
        return self

In [3]:
import tensorflow as tf
import numpy as np

In [38]:
input_size = 10
output_size = 2
hidden_layer_size = 50
tf.reset_default_graph()

In [39]:
inputs = tf.placeholder(tf.float32,[None,input_size])
targets = tf.placeholder(tf.int32,[None,output_size])

In [40]:
weights_1 = tf.get_variable("weights_1",[input_size,hidden_layer_size])
biases_1 = tf.get_variable("biases_1",[hidden_layer_size])
outputs_1 = tf.nn.relu(tf.matmul(inputs,weights_1)+biases_1)

weights_2 = tf.get_variable("weights_2",[hidden_layer_size,hidden_layer_size])
biases_2 = tf.get_variable("biases_2",[hidden_layer_size])
outputs_2 = tf.nn.relu(tf.matmul(outputs_1,weights_2)+biases_2)

weights_3 = tf.get_variable("weights_3",[hidden_layer_size,hidden_layer_size])
biases_3 = tf.get_variable("biases_3",[hidden_layer_size])
outputs_3 = tf.nn.relu(tf.matmul(outputs_2,weights_3)+biases_3)

weights_4 = tf.get_variable("weights_4",[hidden_layer_size,output_size])
biases_4 = tf.get_variable("biases_4",[output_size])
outputs_final = tf.matmul(outputs_3,weights_4)+biases_4

In [41]:
loss = tf.nn.softmax_cross_entropy_with_logits(logits = outputs_final,labels= targets)
mean_loss = tf.reduce_mean(loss)
optimize = tf.train.AdamOptimizer(learning_rate=0.0002).minimize(mean_loss)

In [42]:
out_equals_target = tf.equal(tf.argmax(outputs_final,1),tf.argmax(targets,1))
accuracy = tf.reduce_mean(tf.cast(out_equals_target,tf.float32))

In [43]:
sess = tf.InteractiveSession()
initializer = tf.global_variables_initializer()
sess.run(initializer)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/client/session.py:1735: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [44]:
batch_size = 100
max_epochs = 50
prev_validation_loss = 9999999.

In [45]:
train_data = Audiobooks_Data_Reader('train',batch_size)
validation_data = Audiobooks_Data_Reader('validation')

In [46]:
for i in range(max_epochs):
    curr_epoch_loss = 0.
    for input_batch, target_batch in train_data:
        _,batch_loss = sess.run([optimize,mean_loss],feed_dict={inputs:input_batch,targets:target_batch})
        curr_epoch_loss += batch_loss
    curr_epoch_loss /= train_data.batch_count
    validation_loss = 0.
    validation_accuracy = 0.
    for input_batch, target_batch in validation_data:
        validation_loss,validation_accuracy = sess.run([mean_loss,accuracy],feed_dict={inputs:input_batch,targets:target_batch})
    print ("Epoch: "+str(i+1)+".Training loss:"+"{0:.3f}".format(curr_epoch_loss)+
           ".Validation loss: "+"{0:.3f}".format(validation_loss)+
           ".Validation accuracy: "+"{0:.3f}".format(validation_accuracy*100)+"%")
    if (validation_loss>prev_validation_loss):
        break
    prev_validation_loss = validation_loss
print ("END OF TRAINING!")

Epoch: 1.Training loss:0.658.Validation loss: 0.623.Validation accuracy: 67.114%
Epoch: 2.Training loss:0.597.Validation loss: 0.576.Validation accuracy: 69.799%
Epoch: 3.Training loss:0.552.Validation loss: 0.539.Validation accuracy: 71.588%
Epoch: 4.Training loss:0.515.Validation loss: 0.509.Validation accuracy: 72.260%
Epoch: 5.Training loss:0.484.Validation loss: 0.485.Validation accuracy: 73.378%
Epoch: 6.Training loss:0.459.Validation loss: 0.466.Validation accuracy: 75.168%
Epoch: 7.Training loss:0.439.Validation loss: 0.450.Validation accuracy: 75.168%
Epoch: 8.Training loss:0.423.Validation loss: 0.438.Validation accuracy: 75.392%
Epoch: 9.Training loss:0.410.Validation loss: 0.428.Validation accuracy: 75.392%
Epoch: 10.Training loss:0.400.Validation loss: 0.419.Validation accuracy: 76.510%
Epoch: 11.Training loss:0.391.Validation loss: 0.412.Validation accuracy: 76.957%
Epoch: 12.Training loss:0.383.Validation loss: 0.406.Validation accuracy: 77.629%
Epoch: 13.Training loss:0

In [47]:
test_data = Audiobooks_Data_Reader('test')
test_accuracy = 0.
test_loss = 0.
for input_batch, target_batch in test_data:
    test_accuracy = sess.run([accuracy],feed_dict={inputs:input_batch,targets:target_batch})
print ("Test accuracy: "+"{0:.3f}".format(test_accuracy[0]*100)+"%")

Test accuracy: 80.804%
